In [ ]:
# === RED objects + proportional occlusion + BLUE pedestrian fixed-annulus tracking
#     with in-frame consolidation and multi-ID tracking (delayed numbering)
#     + ID protection radius (cap-to-blue near confirmed IDs)
# + TestSet2.csv with rows only for IDs that have >=5 valid frames
#   Columns: t_ms, id, x, y, vx, vy, v, x_plus_1, y_plus_1
#   x_plus_1/y_plus_1 use the SAME ID at t+1.0s (+4 frames); if missing → 'NaN'

from google.colab import drive, files
import os, re, glob, shutil, csv, zipfile
from datetime import datetime
import numpy as np
import cv2
from sklearn.cluster import DBSCAN
from IPython.display import HTML, display
from collections import defaultdict

# ---------- Config ----------
drive_src    = '/content/drive/MyDrive/RECORDING'
local_src    = '/content/RECORDING_LOCAL'
frames_dir   = '/content/OCCLUSION_FRAMES'
video_path   = '/content/objects_occluded.mp4'
csv_path     = '/content/centroids.csv'      # red objects (with occlusion)
tracks_csv   = '/content/ped_tracks.csv'     # legacy tracker csv (kept)
testset1_csv = '/content/TestSet1.csv'       # NEW final dataset

# Preprocessing
blur_sigma_px = 7

# DBSCAN (meters)
EPS_M          = 0.15
MIN_SAMPLES    = 15
MIN_OBJ_M_RED  = 0.3
MIN_OBJ_M_BLUE = 0.2

# Geometry (y increases downward)
Y_TOP_M, Y_BOTTOM_M = 0.1, 3.5
IGNORE_TOP_M = 1.0

# Persistence / display
MAX_SCORE_CAP  = 10
RED_AT_SCORE   = 5
INCR_ON_HIT    = 1
DECR_ON_MISS   = 1
CLUSTER_USE_VAL_RED = 5
BLUE_SCORE_MIN, BLUE_SCORE_MAX = 1, 4   # ONLY pixels with score < 5 are "blue"

# Cluster-aware dimming
ON_FRAC_HOLD   = 0.60

# Occlusion
OCCLUDED_SKIP_FRAC   = 0.50
OCCLUSION_AREA_RATIO = 3.0

# --- Radii you can tweak ---
ASSOC_MIN_M = 0.00           # allow tiny motion (prevents ID churn when still)
ASSOC_MAX_M = 1.20
RED_BLUE_SUPPRESS_RADIUS_PX = 20
CONSOLIDATION_RADIUS_M      = 1.00
ID_PROTECT_RADIUS_M         = 0.50

# --- Tracking (blue) ---
WARMUP_FRAMES   = 4
CONFIRM_HITS    = 2
MISS_TOL        = 4
DT              = 0.25   # s per frame (=> 250 ms)

# Colors (BGR)
COL_RED_OBJ_PT  = (0, 0, 255)
COL_RED_BBOX    = (0, 255, 0)
COL_OCC_BOX     = (255, 255, 0)
COL_PED         = (255, 0, 0)            # blue dot
COL_PED_TEXT    = (255, 255, 255)        # white text

# ---------- Helpers ----------
pat_full = re.compile(r'^\[(\d{4}-\d{2}-\d{2})\s+(\d{2})-(\d{2})-(\d{2})-(\d{3})\]\.png$')
def extract_dt(fname):
    m = pat_full.match(fname)
    date_str, hh, mm, ss, ms = m.groups()
    return datetime.strptime(f'{date_str} {hh}:{mm}:{ss}.{ms}', '%Y-%m-%d %H:%M:%S.%f')

def oriented_length_m(points_m):
    if points_m.shape[0] < 2: return 0.0
    c = points_m.mean(axis=0)
    _, _, Vt = np.linalg.svd(points_m - c, full_matrices=False)
    axis = Vt[0]
    proj = (points_m - c) @ axis
    return float(proj.max() - proj.min())

def build_color_lut(red_at=5):
    lut = np.zeros((red_at+1, 3), dtype=np.uint8)
    blue = np.array([255, 0, 0], dtype=np.float32)  # B,G,R
    red  = np.array([0, 0, 255], dtype=np.float32)
    for s in range(1, red_at+1):
        t = (s-1) / max(1, (red_at-1))
        lut[s] = np.round(blue*(1-t) + red*t).astype(np.uint8)
    return lut

def rect_contains(outer, inner):
    return (outer[0] <= inner[0] and outer[1] <= inner[1] and
            outer[2] >= inner[2] and outer[3] >= inner[3])

color_lut = build_color_lut(RED_AT_SCORE)

# ---------- Prep input ----------
drive.mount('/content/drive', force_remount=True)
os.makedirs(local_src, exist_ok=True)
for f in glob.glob(os.path.join(drive_src, '*.png')):
    dst = os.path.join(local_src, os.path.basename(f))
    if not os.path.exists(dst):
        shutil.copy2(f, dst)

# Only keep files that exactly match the timestamped naming convention
all_pngs = [f for f in os.listdir(local_src) if f.lower().endswith('.png')]
valid_pngs = [f for f in all_pngs if pat_full.match(f)]
pngs = sorted(valid_pngs, key=extract_dt)
print(f"Using {len(pngs)} PNGs that match the naming convention; "
      f"skipped {len(all_pngs) - len(valid_pngs)} others.")
assert pngs, "No PNGs found that match the [YYYY-MM-DD HH-MM-SS-ms].png pattern."

# ---------- Init ----------
tmp = cv2.imread(os.path.join(local_src, pngs[0]), cv2.IMREAD_GRAYSCALE)
H, W = tmp.shape
SY = (Y_BOTTOM_M - Y_TOP_M) / float(H)
SX = SY  # set SX explicitly if horizontal meters/px differ
center_x = int((W - 1) / 2)

rows_ignore = int(np.ceil(IGNORE_TOP_M / SY))
rows_ignore = max(0, min(H, rows_ignore))

score = np.zeros((H, W), dtype=np.int32)
prev_clusters_pixels = []  # for cluster-aware dimming

os.makedirs(frames_dir, exist_ok=True)

# CSVs (red + legacy track)
with open(csv_path, 'w', newline='') as fcsv:
    writer = csv.writer(fcsv)
    writer.writerow(['frame', 'cluster_idx', 'cx_m', 'cy_m', 'cx_px', 'cy_px'])
with open(tracks_csv, 'w', newline='') as ft:
    tw = csv.writer(ft)
    tw.writerow(['frame','track_id','state','cx_m','cy_m','cx_px','cy_px','speed_mps'])

# Tracking state (multi-ID). ID assigned only when confirmed.
next_track_id = 1
tracks = {}  # key -> dict(...). Use internal keys; assign visible 'id' at confirmation.
internal_next_key = 1

# Suppression kernel
kernel_rad = RED_BLUE_SUPPRESS_RADIUS_PX
dil_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*kernel_rad+1, 2*kernel_rad+1))

# Prev-frame masks for protection exclusion
prev_red_bbox_mask = np.zeros((H, W), dtype=bool)
prev_occ_mask      = np.zeros((H, W), dtype=bool)

# --- Collect per-ID per-frame records for TestSet2, then filter/write at the end ---
# Each record: (frame_idx, t_ms, x, y, vx, vy, v)
records_by_id = defaultdict(list)

# ---------- Process ----------
for frame_idx, fname in enumerate(pngs):
    gray = cv2.imread(os.path.join(local_src, fname), cv2.IMREAD_GRAYSCALE)
    if gray is None:
        print("Skip unreadable:", fname); continue

    # 1) Blur + Otsu
    blurred = cv2.GaussianBlur(gray, (0,0), sigmaX=blur_sigma_px, sigmaY=blur_sigma_px)
    _, mask = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    if rows_ignore > 0: mask[:rows_ignore, :] = 0
    detect = (mask > 0)

    # Cluster-aware dimming → score update
    protected = np.zeros((H, W), dtype=bool)
    for (ys_prev, xs_prev) in prev_clusters_pixels:
        if ys_prev.size and np.mean(detect[ys_prev, xs_prev]) >= ON_FRAC_HOLD:
            protected[ys_prev, xs_prev] = True
    score += INCR_ON_HIT * detect.astype(np.int32) - DECR_ON_MISS * ((~detect) & (~protected)).astype(np.int32)
    score = np.clip(score, 0, MAX_SCORE_CAP)
    if rows_ignore > 0: score[:rows_ignore, :] = 0

    # ID protection radius (cap to 4) except inside last frame's red/occ
    if tracks:
        rad_px = int(max(1, round(ID_PROTECT_RADIUS_M / SY)))
        id_protect_mask = np.zeros((H, W), dtype=np.uint8)
        for tr in tracks.values():
            if tr.get('confirmed', False) and tr.get('last_seen', -999) >= 0 and tr.get('misses', 0) <= MISS_TOL:
                cx_px, cy_px = tr['centroid_px']
                if 0 <= cx_px < W and 0 <= cy_px < H:
                    cv2.circle(id_protect_mask, (int(cx_px), int(cy_px)), rad_px, 1, thickness=-1)
        if id_protect_mask.any():
            block = (id_protect_mask.astype(bool)) & ~(prev_red_bbox_mask | prev_occ_mask)
            score[block] = np.minimum(score[block], RED_AT_SCORE - 1)

    # Color base
    disp = np.minimum(score, RED_AT_SCORE).astype(np.int32)
    base = np.zeros((H, W, 3), dtype=np.uint8); nz = disp > 0
    base[nz] = color_lut[disp[nz]]
    annotated = base.copy()

    # ---------- RED OBJECTS ----------
    stable_red = (score >= CLUSTER_USE_VAL_RED)
    ys_r, xs_r = np.where(stable_red)
    clusters_r = []
    if xs_r.size:
        x_m_r = (xs_r - center_x) * SX
        y_m_r = Y_TOP_M + ys_r * SY
        pts_r = np.c_[x_m_r, y_m_r]
        db_r = DBSCAN(eps=EPS_M, min_samples=MIN_SAMPLES).fit(pts_r)
        labels_r = db_r.labels_; uniq_r = [lab for lab in set(labels_r) if lab != -1]
        for lab in uniq_r:
            idx = np.where(labels_r == lab)[0]
            if oriented_length_m(pts_r[idx]) < MIN_OBJ_M_RED: continue
            xk, yk = xs_r[idx], ys_r[idx]
            x0, x1 = int(xk.min()), int(xk.max()); y0, y1 = int(yk.min()), int(yk.max())
            clusters_r.append(dict(idx=idx, xk=xk, yk=yk, x0=x0, x1=x1, y0=y0, y1=y1))
    clusters_r.sort(key=lambda c: c['y0'])

    # Nested red suppression (bbox)
    kept, keep_rects = [], []
    for c in clusters_r:
        bbox = (c['x0'], c['y0'], c['x1'], c['y1'])
        if any(rect_contains(R, bbox) for R in keep_rects): continue
        kept.append(c); keep_rects.append(bbox)
    clusters_r = kept

    # Proportional occlusion rects
    def compute_occ(c):
        x0, x1, y0, y1 = c['x0'], c['x1'], c['y0'], c['y1']
        bbox_w = (x1 - x0 + 1); bbox_h = (y1 - y0 + 1)
        bbox_area_px = bbox_w * bbox_h
        scale = np.sqrt(OCCLUSION_AREA_RATIO)
        desired_w = max(bbox_w, int(np.ceil(scale * bbox_w)))
        desired_h = max(bbox_h, int(np.ceil(scale * bbox_h)))
        height_limit = H - y0
        if x0 > center_x:
            width_limit = W - x0
            occ_h = min(desired_h, height_limit); occ_w = min(desired_w, width_limit)
            target_area = int(np.ceil(OCCLUSION_AREA_RATIO * bbox_area_px))
            area = occ_w * occ_h
            if area < target_area and occ_h > 0:
                if occ_w < width_limit:
                    occ_w = min(width_limit, max(occ_w, int(np.ceil(target_area / occ_h)))); area = occ_w * occ_h
                if area < target_area and occ_h < height_limit and occ_w > 0:
                    occ_h = min(height_limit, max(occ_h, int(np.ceil(target_area / occ_w))))
            occ_h = max(min(occ_h, height_limit), bbox_h); occ_w = min(occ_w, width_limit)
            occ_x0 = x0; occ_x1 = min(W-1, x0 + occ_w - 1)
        elif x1 < center_x:
            width_limit = x1 + 1
            occ_h = min(desired_h, height_limit); occ_w = min(desired_w, width_limit)
            target_area = int(np.ceil(OCCLUSION_AREA_RATIO * bbox_area_px))
            area = occ_w * occ_h
            if area < target_area and occ_h > 0:
                if occ_w < width_limit:
                    occ_w = min(width_limit, max(occ_w, int(np.ceil(target_area / occ_h)))); area = occ_w * occ_h
                if area < target_area and occ_h < height_limit and occ_w > 0:
                    occ_h = min(height_limit, max(occ_h, int(np.ceil(target_area / occ_w))))
            occ_h = max(min(occ_h, height_limit), bbox_h); occ_w = min(occ_w, width_limit)
            occ_x1 = x1; occ_x0 = max(0, x1 - occ_w + 1)
        else:
            occ_x0, occ_x1 = x0, x1; occ_h = min(H - y0, desired_h)
        occ_y0 = y0; occ_y1 = min(H-1, y0 + occ_h - 1)
        c.update(dict(occ_x0=occ_x0, occ_x1=occ_x1, occ_y0=occ_y0, occ_y1=occ_y1))
    for c in clusters_r: compute_occ(c)

    # Nested suppression again with occ
    kept2, keep_rects = [], []
    for c in clusters_r:
        bbox = (c['x0'], c['y0'], c['x1'], c['y1'])
        occ  = (c['occ_x0'], c['occ_y0'], c['occ_x1'], c['occ_y1'])
        if any(rect_contains(R, bbox) or rect_contains(R, occ) for R in keep_rects): continue
        kept2.append(c); keep_rects += [bbox, occ]
    clusters_r = kept2

    # Masks where blue cannot exist as IDs
    red_bbox_mask = np.zeros((H, W), dtype=bool)
    occ_mask_pre  = np.zeros((H, W), dtype=bool)
    for c in clusters_r:
        red_bbox_mask[c['y0']:c['y1']+1, c['x0']:c['x1']+1] = True
        ox0, oy0, ox1, oy1 = c['occ_x0'], c['occ_y0'], c['occ_x1'], c['occ_y1']
        occ_mask_pre[oy0:oy1+1, ox0:ox1+1] = True

    # Remember current red/occ regions for the NEXT frame’s ID protection exclusion
    prev_red_bbox_mask = red_bbox_mask.copy()
    prev_occ_mask      = occ_mask_pre.copy()

    # ---------- Red→Blue radius suppression ----------
    blue_mask = (score >= BLUE_SCORE_MIN) & (score <= BLUE_SCORE_MAX)
    red_all_mask = (score >= CLUSTER_USE_VAL_RED)
    red_influence = cv2.dilate(red_all_mask.astype(np.uint8), dil_kernel, 1).astype(bool)
    suppressed_blue = blue_mask & red_influence
    allowed_blue = blue_mask & ~suppressed_blue & ~red_bbox_mask & ~occ_mask_pre
    annotated[suppressed_blue] = (0, 0, 0)

    # ---------- Draw RED (fill then outlines) ----------
    occluded_mask = np.zeros((H, W), dtype=bool)
    new_prev_clusters_pixels = []
    for c in clusters_r:
        xk, yk = c['xk'], c['yk']
        if xk.size and np.mean(occluded_mask[yk, xk]) >= OCCLUDED_SKIP_FRAC:
            new_prev_clusters_pixels.append((yk.copy(), xk.copy()))
            continue
        x0, x1, y0, y1 = c['x0'], c['x1'], c['y0'], c['y1']
        occ_x0, occ_x1, occ_y0, occ_y1 = c['occ_x0'], c['occ_x1'], c['occ_y0'], c['occ_y1']
        annotated[occ_y0:occ_y1+1, occ_x0:occ_x1+1] = (0, 0, 0)
        annotated[yk, xk] = COL_RED_OBJ_PT
        occluded_mask[occ_y0:occ_y1+1, occ_x0:occ_x1+1] = True
        new_prev_clusters_pixels.append((yk.copy(), xk.copy()))
    prev_clusters_pixels = new_prev_clusters_pixels

    with open(csv_path, 'a', newline='') as fcsv:
        writer = csv.writer(fcsv)
        for k, c in enumerate(clusters_r, start=1):
            xk, yk = c['xk'], c['yk']
            x0, x1, y0, y1 = c['x0'], c['x1'], c['y0'], c['y1']
            occ_x0, occ_x1, occ_y0, occ_y1 = c['occ_x0'], c['occ_x1'], c['occ_y0'], c['occ_y1']
            cx_px = int(np.round(xk.mean())); cy_px = int(np.round(yk.mean()))
            cx_m = (cx_px - center_x) * SX; cy_m = Y_TOP_M + cy_px * SY
            cv2.rectangle(annotated, (occ_x0, occ_y0), (occ_x1, occ_y1), COL_OCC_BOX, 1)
            cv2.rectangle(annotated, (x0, y0), (x1, y1), COL_RED_BBOX, 1)
            cv2.circle(annotated, (cx_px, cy_px), 3, (0, 255, 0), -1)
            cv2.putText(annotated, f"x={cx_m:.2f}m, y={cy_m:.2f}m",
                        (cx_px + 5, max(10, cy_px - 5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
            writer.writerow([fname, k, f"{cx_m:.3f}", f"{cy_m:.3f}", cx_px, cy_px])

    # ---------- BLUE detections ----------
    ys_b, xs_b = np.where(allowed_blue)
    detections = []
    if xs_b.size:
        x_m_b = (xs_b - center_x) * SX
        y_m_b = Y_TOP_M + ys_b * SY
        pts_b = np.c_[x_m_b, y_m_b]
        db_b = DBSCAN(eps=EPS_M, min_samples=MIN_SAMPLES).fit(pts_b)
        labels_b = db_b.labels_; uniq_b = [lab for lab in set(labels_b) if lab != -1]
        for lab in uniq_b:
            idx = np.where(labels_b == lab)[0]
            if oriented_length_m(pts_b[idx]) < MIN_OBJ_M_BLUE: continue
            xk, yk = xs_b[idx], ys_b[idx]
            cx_px = int(np.round(xk.mean())); cy_px = int(np.round(yk.mean()))
            cx_m = (cx_px - center_x) * SX; cy_m = Y_TOP_M + cy_px * SY
            if red_bbox_mask[min(H-1, max(0, cy_px)), min(W-1, max(0, cx_px))]: continue
            if occ_mask_pre[min(H-1, max(0, cy_px)), min(W-1, max(0, cx_px))]: continue
            detections.append(dict(
                cx_m=cx_m, cy_m=cy_m, cx_px=cx_px, cy_px=cy_px,
                n=idx.size, extent_m=oriented_length_m(pts_b[idx])
            ))

    # ---------- In-frame consolidation ----------
    if len(detections) > 1:
        coords = np.array([[d['cx_m'], d['cy_m']] for d in detections], dtype=float)
        diff = coords[:, None, :] - coords[None, :, :]
        D = np.sqrt(np.sum(diff*diff, axis=2))
        adjacency = (D <= CONSOLIDATION_RADIUS_M)
        N = len(detections)
        visited = np.zeros(N, dtype=bool)
        keep_idx = []
        for i in range(N):
            if visited[i]: continue
            stack = [i]; group = []
            while stack:
                j = stack.pop()
                if visited[j]: continue
                visited[j] = True
                group.append(j)
                for nb in np.where(adjacency[j])[0]:
                    if not visited[nb]:
                        stack.append(nb)
            best = max(group, key=lambda g: (detections[g]['n'], detections[g]['extent_m'], -g))
            keep_idx.append(best)
        detections = [detections[k] for k in sorted(keep_idx)]

    # ---------- Tracking ----------
    matched_det_idx = set()
    matched_track_keys = set()

    if frame_idx >= WARMUP_FRAMES and (tracks or detections):
        track_items = list(tracks.items())
        pairs = []
        for ti, (k, tr) in enumerate(track_items):
            gap = max(1, frame_idx - tr['last_seen'])
            for di, det in enumerate(detections):
                d = float(np.hypot(det['cx_m'] - tr['centroid_m'][0], det['cy_m'] - tr['centroid_m'][1]))
                if ASSOC_MIN_M <= d <= ASSOC_MAX_M:
                    pairs.append((d, ti, di, gap))
        pairs.sort(key=lambda x: x[0])

        used_tracks = set()
        used_dets = set()
        for d, ti, di, gap in pairs:
            if ti in used_tracks or di in used_dets:
                continue
            key, tr = track_items[ti]
            used_tracks.add(ti); used_dets.add(di)
            matched_det_idx.add(di); matched_track_keys.add(key)

            prev_m = tr['centroid_m']
            new_m  = np.array([detections[di]['cx_m'], detections[di]['cy_m']], dtype=float)

            dt_sec = DT * max(1, gap)
            vx = (new_m[0] - prev_m[0]) / dt_sec
            vy = (new_m[1] - prev_m[1]) / dt_sec
            disp = float(np.linalg.norm(new_m - prev_m))
            speed = disp / dt_sec

            tr.update({
                'centroid_m': new_m,
                'centroid_px': (detections[di]['cx_px'], detections[di]['cy_px']),
                'last_seen': frame_idx,
                'misses': 0,
                'hits': tr['hits'] + 1,
                'prev_m': prev_m,
                'speed': speed,
                'vel_mps': (vx, vy)
            })
            if not tr['confirmed'] and tr['hits'] >= CONFIRM_HITS:
                tr['confirmed'] = True
                tr['id'] = next_track_id
                next_track_id += 1

        # Unmatched tracks → miss
        for key, tr in list(tracks.items()):
            if key not in matched_track_keys:
                tr['misses'] += 1
                if tr['misses'] > MISS_TOL:
                    del tracks[key]

        # New tracks from unmatched detections
        for di, det in enumerate(detections):
            if di in matched_det_idx: continue
            tracks[('t', internal_next_key)] = dict(
                centroid_m=np.array([det['cx_m'], det['cy_m']], dtype=float),
                centroid_px=(det['cx_px'], det['cy_px']),
                last_seen=frame_idx,
                hits=1, misses=0, confirmed=False,
                id=None,
                prev_m=np.array([det['cx_m'], det['cy_m']], dtype=float),
                speed=0.0,
                vel_mps=(0.0, 0.0)
            )
            internal_next_key += 1

    else:
        # Warm-up: seed tentative tracks (no rendering)
        for det in detections:
            tracks[('t', internal_next_key)] = dict(
                centroid_m=np.array([det['cx_m'], det['cy_m']], dtype=float),
                centroid_px=(det['cx_px'], det['cy_px']),
                last_seen=frame_idx,
                hits=1, misses=0, confirmed=False,
                id=None,
                prev_m=np.array([det['cx_m'], det['cy_m']], dtype=float),
                speed=0.0,
                vel_mps=(0.0, 0.0)
            )
            internal_next_key += 1

    # --- Draw confirmed tracks + append to histories (NOT writing TestSet2 here) ---
    with open(tracks_csv, 'a', newline='') as ft:
        tw = csv.writer(ft)
        for key, tr in tracks.items():
            if not tr['confirmed'] or tr['last_seen'] != frame_idx:
                continue

            cx_px, cy_px = tr['centroid_px']
            # Skip if centroid inside red bbox or occlusion
            if red_bbox_mask[min(H-1, max(0, cy_px)), min(W-1, max(0, cx_px))]: continue
            if occ_mask_pre[min(H-1, max(0, cy_px)), min(W-1, max(0, cx_px))]: continue

            cx_m, cy_m = tr['centroid_m']
            vx, vy = tr.get('vel_mps', (0.0, 0.0))
            v = float(np.hypot(vx, vy))

            # Dot
            cv2.circle(annotated, (int(cx_px), int(cy_px)), 3, COL_PED, -1)

            # Label block (ID + per-line metrics)
            id_scale  = 0.60
            num_scale = max(0.18, id_scale / 3.0)  # ~1/3 of ID size
            base_x = int(cx_px) + 6
            base_y = max(12, int(cy_px) - 6)
            line_step = 12

            cv2.putText(annotated, f"ID {tr['id']}",
                        (base_x, base_y),
                        cv2.FONT_HERSHEY_SIMPLEX, id_scale, COL_PED_TEXT, 1, cv2.LINE_AA)
            cv2.putText(annotated, f"x={cx_m:.2f} m",
                        (base_x, base_y + 1*line_step),
                        cv2.FONT_HERSHEY_SIMPLEX, num_scale, COL_PED_TEXT, 1, cv2.LINE_AA)
            cv2.putText(annotated, f"y={cy_m:.2f} m",
                        (base_x, base_y + 2*line_step),
                        cv2.FONT_HERSHEY_SIMPLEX, num_scale, COL_PED_TEXT, 1, cv2.LINE_AA)
            cv2.putText(annotated, f"vx={vx:+.2f} m/s",
                        (base_x, base_y + 3*line_step),
                        cv2.FONT_HERSHEY_SIMPLEX, num_scale, COLPED_TEXT:=COL_PED_TEXT, 1, cv2.LINE_AA)  # small trick to keep color const
            cv2.putText(annotated, f"vy={vy:+.2f} m/s",
                        (base_x, base_y + 4*line_step),
                        cv2.FONT_HERSHEY_SIMPLEX, num_scale, COL_PED_TEXT, 1, cv2.LINE_AA)
            cv2.putText(annotated, f"v={v:.2f} m/s",
                        (base_x, base_y + 5*line_step),
                        cv2.FONT_HERSHEY_SIMPLEX, num_scale, COL_PED_TEXT, 1, cv2.LINE_AA)

            # Legacy tracker csv (kept)
            state = ''
            tw.writerow([fname, tr['id'], state, f"{cx_m:.3f}", f"{cy_m:.3f}",
                         int(cx_px), int(cy_px), f"{tr['speed']:.3f}"])

            # Append to in-memory history for TestSet2
            t_ms = int(round((frame_idx + 1) * DT * 1000))  # 250, 500, 750, ...
            records_by_id[tr['id']].append(
                (frame_idx, t_ms, float(cx_m), float(cy_m), float(vx), float(vy), float(v))
            )

    # Save frame
    out = os.path.join(frames_dir, f"{os.path.splitext(fname)[0]}_occ.png")
    cv2.imwrite(out, annotated)

print(f"\nSaved {len(os.listdir(frames_dir))} frames to {frames_dir}")
print(f"Centroids CSV → {csv_path}")
print(f"Pedestrian tracks CSV → {tracks_csv}")

# ---------- Build TestSet2.csv at the end (filter IDs with >=5 frames; x+1/y+1 need exact t+1.0s row) ----------
with open(testset1_csv, 'w', newline='') as fts1:
    w2 = csv.writer(fts1)
    w2.writerow(['t_ms','id','x','y','vx','vy','v','x_plus_1','y_plus_1'])

    for tid, recs in records_by_id.items():
        # Only include IDs with at least 5 valid frames
        if len(recs) < 5:
            continue
        # Map frame_idx -> (x,y) to fetch exact +4 frames
        by_frame = {fi: (x, y) for (fi, _t, x, y, _vx, _vy, _v) in recs}
        # Emit rows
        for (fi, t_ms, x, y, vx, vy, v) in recs:
            target_fi = fi + 4  # +1.0 s at 4 Hz
            if target_fi in by_frame:
                x1, y1 = by_frame[target_fi]
                x1s, y1s = f"{x1:.6f}", f"{y1:.6f}"
            else:
                x1s, y1s = "NaN", "NaN"
            w2.writerow([t_ms, tid,
                         f"{x:.6f}", f"{y:.6f}",
                         f"{vx:.6f}", f"{vy:.6f}", f"{v:.6f}",
                         x1s, y1s])

print(f"TestSet1 CSV → {testset1_csv}")

# ---------- Build MP4 @ 4 Hz ----------
fps = 4
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(video_path, fourcc, fps, (W, H), True)
for fname in pngs:
    img_path = os.path.join(frames_dir, f"{os.path.splitext(fname)[0]}_occ.png")
    frame = cv2.imread(img_path)
    if frame is not None:
        writer.write(frame)
writer.release()

print(f"\nWrote video → {video_path} at {fps} fps ({W}x{H}).")
display(HTML(f'<video controls src="{video_path}" width="640"></video>'))
files.download(video_path)
files.download(csv_path)
files.download(tracks_csv)
files.download(testset1_csv)

# ---------- Zip all final annotated frames ----------
zip_path = '/content/annotated_frames.zip'
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for fn in sorted(os.listdir(frames_dir)):
        if fn.endswith('_occ.png'):
            zf.write(os.path.join(frames_dir, fn), arcname=fn)

# NEW: copy the ZIP to Google Drive (keeps everything else the same)
drive_zip_path = '/content/drive/MyDrive/annotated_frames.zip'
shutil.copy2(zip_path, drive_zip_path)
print(f"Saved annotated frames ZIP to Drive → {drive_zip_path}")

files.download(zip_path)


Mounted at /content/drive
Using 4241 PNGs that match the naming convention; skipped 0 others.

Saved 4241 frames to /content/OCCLUSION_FRAMES
Centroids CSV → /content/centroids.csv
Pedestrian tracks CSV → /content/ped_tracks.csv
TestSet1 CSV → /content/TestSet1.csv

Wrote video → /content/objects_occluded.mp4 at 4 fps (600x350).


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved annotated frames ZIP to Drive → /content/drive/MyDrive/annotated_frames.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>